### Consumindo os dados da API de localidades

API de localidades: https://servicodados.ibge.gov.br/api/docs/localidades#api-_


In [ ]:
import pandas as pd
import requests
import json

In [ ]:
import sqlalchemy as db
import mysql.connector
from sqlalchemy import create_engine
from pandasql import PandaSQL
import pymysql

In [ ]:
UFs = [11,12,13,14,15,16,17,21,22,23,24,25,26,27,28,29,31,32,33,35,41,42,43,50,51,52,53]

dfs = []

for UF in UFs:
    url = f"https://servicodados.ibge.gov.br/api/v1/localidades/estados/{UF}/distritos"
    dados = requests.get(url).text
    distritos = json.loads(dados)
    dados_norm = pd.json_normalize(distritos)
    dfs.append(dados_norm)
    
todos = pd.concat(dfs, ignore_index=True)
todos

In [ ]:
todos.columns = todos.columns.str.replace('.', '_')
todos.columns = todos.columns.str.replace('-', '_')

Estabelecendo a conexão com o banco de dados MySQL:

In [ ]:
engine = create_engine("mysql+pymysql://root:@localhost:3306/test")
connection = engine.connect()
pdsql = PandaSQL()

Lendo os dados de cada UF:

In [ ]:
def read_item(UF_id: int):
    query_uf_id = f"SELECT * FROM testeapi WHERE municipio_microrregiao_mesorregiao_UF_id = {UF_id};"
    df_uf_id = pd.read_sql(query_uf_id, connection)
    json_uf_id = df_uf_id.to_json(orient = 'records')
    result_id = json.loads(json_uf_id)
    return result_id

In [ ]:
# Exemplo São Paulo: UF = 35

read_item(35)